In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

# Load corpus
data_dir = "/content/drive/MyDrive/AML_project/Mental_disorders"
os.listdir(data_dir)

corpus = []

for filename in os.listdir(data_dir):
    print(f"Loading file {filename}")

    with open(f"{data_dir}/{filename}") as f:
        lines = f.read().split('\n')
        for line in lines:
          corpus.append(line)

print(len(corpus))

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Loading file Schizophrenia_Wikipedia
Loading file BipolarDisorder_Wikipedia
Loading file EatingDisorder_Wikipedia
Loading file MoodDisorder_Wikipedia
Loading file AnxietyDisorder_Wikipedia
Loading file PersonalityDisorder_Wikipedia
Loading file ObsessiveCompulsiveDisorder_Wikipedia
Loading file Post-traumaticStressDisorder_Wikipedia
Loading file Psychosis_Wikipedia
Loading file Depression_Wikipedia
Loading file PanicDisorder_Wikipedia
Loading file SpecificPhobia_Wikipedia
934


In [ ]:
# Build Embeddings
embeddings = model.encode(corpus)
assert embeddings.shape[0] == len(corpus)

In [ ]:
try:
    import faiss
except:
    !pip install faiss-cpu -q
    import faiss


# Build the index
d = embeddings.shape[1]        # Dimension
index = faiss.IndexFlatL2(d)

# Add vectors to the index
index.add(embeddings)
print(index.ntotal)

# Sanity Check
# "model" is our Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')
xq = model.encode(["What is depression?"])
k = 1
D, I = index.search(xq, k)

print(corpus[I[0][0]])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 71.5 MB/s eta 0:00:00
934
Depression is a mental state of low mood and aversion to activity.[3] It affects more than 280 million people of all ages (about 3.5% of the global population).[4] Depression affects a person's thoughts, behavior, feelings, and sense of well-being.[5] Depressed people often experience loss of motivation or interest in, or reduced pleasure or joy from, experiences that would normally bring them pleasure or joy.[6] Depressed mood is a symptom of some mood disorders such as major depressive disorder and dysthymia;[7] it is a normal temporary reaction to life events, such as the loss of a loved one; and it is also a symptom of some physical diseases and a side effect of some drugs and medical treatments.


In [ ]:
try:
    import gpt4all
except:
    !pip install gpt4all==0.3.6 -q
    import gpt4all

# gptj = gpt4all.GPT4All("ggml-gpt4all-j-v1.3-groovy")

gptj = gpt4all.GPT4All("orca-mini-7b.ggmlv3.q4_0.bin")

# Nearest neighbour from corpus as per FAISS
text = corpus[I[0][0]]

messages = [
    {
        "role": "user",
        "content": (
            "Take the context from context xml tags and answer the question "
            f" between question xml tags. <context>{text}<context>"
            "<question>What is depression?<question>"
        )
    }
]
response = gptj.chat_completion(messages)

print(response["choices"][0]["message"]["content"])

Found model file at  /root/.cache/gpt4all/orca-mini-7b.ggmlv3.q4_0.bin
### Instruction: 
            The prompt below is a question to answer, a task to complete, or a conversation 
            to respond to; decide which and write an appropriate response.
            
### Prompt: 
Take the context from context xml tags and answer the question  between question xml tags. <context>Depression is a mental state of low mood and aversion to activity.[3] It affects more than 280 million people of all ages (about 3.5% of the global population).[4] Depression affects a person's thoughts, behavior, feelings, and sense of well-being.[5] Depressed people often experience loss of motivation or interest in, or reduced pleasure or joy from, experiences that would normally bring them pleasure or joy.[6] Depressed mood is a symptom of some mood disorders such as major depressive disorder and dysthymia;[7] it is a normal temporary reaction to life events, such as the loss of a loved one; and it is also

In [ ]:
def QnA(question):
  xq = model.encode([question])
  k = 1
  D, I = index.search(xq, k)

  text = corpus[I[0][0]]

  messages = [
      {
          "role": "user",
          "content": (
              "Take the context from context xml tags and answer the question "
              f" between question xml tags. <context>{text}<context>"
              f"<question>{question}<question>"
          )
      }
  ]
  response = gptj.chat_completion(messages)

  print(response["choices"][0]["message"]["content"])

In [ ]:
question = "What steps should a person diagnosed with depression take?"
QnA(question)

### Instruction: 
            The prompt below is a question to answer, a task to complete, or a conversation 
            to respond to; decide which and write an appropriate response.
            
### Prompt: 
Take the context from context xml tags and answer the question  between question xml tags. <context>Depression is a major mental-health cause of disease burden. Its consequences further lead to significant burden in public health, including a higher risk of dementia, premature mortality arising from physical disorders, and maternal depression impacts on child growth and development.[61] Approximately 76% to 85% of depressed people in low- and middle-income countries do not receive treatment;[62] barriers to treatment include: inaccurate assessment, lack of trained health-care providers, social stigma and lack of resources.[4]<context><question>What is depression?<question>
### Response:
 
 <response>Depression is a mental state of low mood and aversion to activity. It affects m

In [ ]:
question = "What are the symptoms of Schizophrenia?"
QnA(question)

### Instruction: 
            The prompt below is a question to answer, a task to complete, or a conversation 
            to respond to; decide which and write an appropriate response.
            
### Prompt: 
Take the context from context xml tags and answer the question  between question xml tags. <context>Schizophrenia is a mental disorder[14] characterized by continuous or relapsing episodes of psychosis.[5] Major symptoms include hallucinations (typically hearing voices), delusions and disorganized thinking.[7] Other symptoms include social withdrawal and flat affect.[5] Symptoms typically develop gradually, begin during young adulthood, and in many cases are never resolved.[3][7] There is no objective diagnostic test; diagnosis is based on observed behavior, a psychiatric history that includes the person's reported experiences, and reports of others familiar with the person.[7] For a diagnosis of schizophrenia, the described symptoms need to have been present for at least six m

In [ ]:
question = "How is Eating Disorder diagnosed?"
QnA(question)

### Instruction: 
            The prompt below is a question to answer, a task to complete, or a conversation 
            to respond to; decide which and write an appropriate response.
            
### Prompt: 
Take the context from context xml tags and answer the question  between question xml tags. <context>According to Pritts and Susman "The medical history is the most powerful tool for diagnosing eating disorders".[226] There are many medical disorders that mimic eating disorders and comorbid psychiatric disorders. Early detection and intervention can assure a better recovery and can improve a lot the quality of life of these patients. In the past 30 years eating disorders have become increasingly conspicuous and it is uncertain whether the changes in presentation reflect a true increase.[227] Anorexia nervosa and bulimia nervosa are the most clearly defined subgroups of a wider range of eating disorders. Many patients present with subthreshold expressions of the two main diagnose

In [ ]:
question = "What are some extreme cases of Obsessive Compulsive Disorder (OCD)?"
QnA(question)

### Instruction: 
            The prompt below is a question to answer, a task to complete, or a conversation 
            to respond to; decide which and write an appropriate response.
            
### Prompt: 
Take the context from context xml tags and answer the question  between question xml tags. <context>OCD is sometimes placed in a group of disorders called the obsessive–compulsive spectrum.[163]<context><question>What is depression?<question>
### Response:
 
 <response>OCD is sometimes placed in a group of disorders called the obsessive–compulsive spectrum. It is characterized by the presence of repetitive thoughts or compulsions that are difficult to control. These thoughts or compulsions often interfere with daily life and can be distressing. While OCD is not the same as depression, it can sometimes be misdiagnosed as a mood disorder. It is important to differentiate between the two disorders and to consider other potential causes of symptoms, such as medical conditions or su

In [ ]:
question = "How to take care of a person diagnosed with Bipolar Disorder?"
QnA(question)

### Instruction: 
            The prompt below is a question to answer, a task to complete, or a conversation 
            to respond to; decide which and write an appropriate response.
            
### Prompt: 
Take the context from context xml tags and answer the question  between question xml tags. <context>The aim of management is to treat acute episodes safely with medication and work with the patient in long-term maintenance to prevent further episodes and optimise function using a combination of  pharmacological and psychotherapeutic techniques.[12] Hospitalization may be required especially with the manic episodes present in bipolar I. This can be voluntary or (local legislation permitting) involuntary. Long-term inpatient stays are now less common due to deinstitutionalization, although these can still occur.[124] Following (or in lieu of) a hospital admission, support services available can include drop-in centers, visits from members of a community mental health team or an A